# TP04 MapReduce - Multiplication Matricielle
# Matrice A et B complètes

## Chargement des matrices A et B

In [1]:
import numpy as np
import time
from concurrent.futures import ProcessPoolExecutor
from concurrent.futures import ThreadPoolExecutor
import matplotlib.pyplot as plt

In [2]:
A = np.load("A.npy")
B = np.load("B.npy")

# Affichage des dimensions pour vérification
print(f"Matrice A : {A.shape}")
print(f"Matrice B : {B.shape}")

Matrice A : (100, 10000)
Matrice B : (10000, 100)


In [3]:
C_numpy = A @ B

## Questions

1. Ecrire un algorithme MapReduce qui calcule le produit matriciel de A et B en **multithreading** dans lequel chaque worker réalise le calcul d’un ou plusieurs $c_{ij}$

In [ ]:
def compute_element(args):
    """ Calcule un élément c_ij du produit matriciel C = A * B """
    A, B, i, j = args
    return i, j, sum(A[i, k] * B[k, j] for k in range(A.shape[1]))

def Produit_mt(A,B):
    C = np.zeros((A.shape[0], B.shape[1]))

    # Liste des tâches à répartir entre les workers
    tasks = [(A, B, i, j) for i in range(A.shape[0]) for j in range(B.shape[1])]

    # Liste des tâches à répartir entre les workers
    tic = time.time()
    with ThreadPoolExecutor(4) as t_exe:
        results = t_exe.map(compute_element, tasks)
    toc = time.time()

    print(results)
    # Remplissage de la matrice résultat
    for i, j, value in results:
        C[i, j] = value

    print(f"{(toc - tic):.2f} seconds for MT")
    return C


In [21]:
C_mt = Produit_mt(A,B)

64.57 seconds for MT


In [20]:
%time C_mt = Produit_mt(A,B)

65.21 seconds for MT
CPU times: total: 13.8 s
Wall time: 1min 5s


2. Ecrire un algorithme MapReduce qui calcule le produit matriciel de A et B en **mutiprocessing** dans lequel chaque worker réalise le calcul d’un ou plusieurs $c_{ij}$

In [18]:
def Produit_mp(A, B):
    
    C = np.zeros((A.shape[0], B.shape[1]))
    
    # Liste des tâches à répartir entre les workers
    tasks = [(A, B, i, j) for i in range(A.shape[0]) for j in range(B.shape[1])]
    
    # Utilisation de Pool pour le parallélisme
    tic = time.time()
    with ProcessPoolExecutor(max_workers=4) as p_exe:
        results = p_exe.map(compute_element, tasks)
    toc = time.time()
    
    print(results)
    # Remplissage de la matrice résultat
    for i, j, value in results:
        C[i, j] = value
    
    print(f"{(toc - tic):.2f} seconds for MP")
    return C

In [19]:
%time C_mp = Produit_mp(A,B)

<generator object _chain_from_iterable_of_lists at 0x000001E993B3B780>


BrokenProcessPool: A process in the process pool was terminated abruptly while the future was running or pending.

3.Comparer les temps de restitution des différentes implémentations en variant le nombre de workers.

In [12]:
#TODO